# KEA

Author: Moshe C. Silverstein  
Date: 07-2017

In [1]:
import sys
import numpy as np
import pandas as pd
import Utilities
from Utilities import *
from importlib import reload

# Load Data

Data Retrieved 2/1/2017 

http://amp.pharm.mssm.edu/X2K/datasets/KEA.zip (KEA)

In [2]:
kea_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Input/KEA/phosphorylation_reactions.csv', sep=',', header=None)

# Load Gmene Mapping File

In [3]:
mappingFile = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Doc and Mapping/mappingFile_2017.txt', sep='\t', header=None, index_col=0)

# Select Relevent Data

In [4]:
kea_df = kea_df[[2,3,4,5]]

In [5]:
kea_df.rename(columns={2: 'Protein A (gene name)', 3:'Protein B (gene name)', 4:'PubMed ID', 5:'Source databases'}, inplace=True)

In [6]:
# change dataset so each pubmed id is associated with one interaction
lstPub = []
lstPA = []
lstPB = []

for i,index in enumerate(kea_df.index):
    
    progressPercent = ((i+1)/len(kea_df.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(kea_df.index)))
    sys.stdout.flush()
    
    for pub in kea_df.ix[index, 'PubMed ID'].split(';'):
        lstPub.append(pub)
        lstPA.append(kea_df.ix[index, 'Protein A (gene name)'])
        lstPB.append(kea_df.ix[index, 'Protein B (gene name)'])

kea_df_better = pd.DataFrame()
kea_df_better['Protein A (gene name)'] = lstPA
kea_df_better['Protein B (gene name)'] = lstPB
kea_df_better['PubMed ID'] = lstPub

In [7]:
kea_df = kea_df_better.copy()

# Map Gene to human and updated approved symbols

In [8]:
mapgenesymbols(kea_df, mappingFile, 'Protein A (gene name)', 'Protein B (gene name)')

# Unfiltered

In [9]:
# uf = unfiltered
kea_uf_df = kea_df.copy()

### Combine duplicate ppis while concatanatig referances

In [10]:
combineDupPPIs(kea_uf_df)

### Create .sig File

In [11]:
pathU = '~/./Documents/Harmonizome/PPI Library/Output/IndividualResources/Unfiltered/'

createSigFile(kea_uf_df, pathU, 'kea', False)

# Filtered

In [12]:
kea_f_df = kea_df.copy()

In [13]:
# drop any data that was published with more then 10 PPI's per publication or doesnt have a ppi
filterPPIbyPubmed(kea_f_df, 10)

### Combine duplicate ppis while concatanatig referances

In [14]:
combineDupPPIs(kea_f_df)

### Create .sig File

In [15]:
pathF = '~/./Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/'

createSigFile(kea_f_df, pathF, 'kea', True)

## Create .gmt File

Code Below Written By: Marina Latif

In [10]:
# Download both filtered and unfiltered SIG files

#Download UNFILTERED data
#kea_sig = pd.read_table("~/Desktop/Projects/KEA3/PPI/KEA/kea_unfiltered_ppi_2017_07_12.sig", header = None)

#Download FILTERED data
kea_sig = pd.read_table("~/Desktop/Projects/KEA3/PPI/KEA/kea_filtered_ppi_2017_07_12.sig", header = None)

In [11]:
#convert a SIG file to a GMT file
#Create two versions of df 'd1' --> one flipped and one which is in original order
d1 = {'protein_1': kea_sig[0], 'protein_2': kea_sig[5]}
df = pd.DataFrame(data = d1)
df.drop_duplicates(inplace = True)

d2 = {'protein_1': kea_sig[5], 'protein_2': kea_sig[0]}
df_flipped = pd.DataFrame(data = d2)
df_flipped.drop_duplicates(inplace = True)

#Make 'df' and 'df_flipped' into a single frame to account for opposite interactions
extend = pd.concat([df, df_flipped])

#Drop any duplicates and set index according to protein 1 so that we can aggregate 
#all interacting protein 2's by a shared interaction with protein one
extend.drop_duplicates(inplace = True)
extend.set_index('protein_1', inplace = True)

gmt = extend.groupby('protein_1').agg(lambda x: tuple(x))

# Create column representing counts of protein interactions per protein
gmt['interactions'] = [int(len(lst)) for protein, lst in gmt['protein_2'].iteritems()]

# Sort proteins from max to min according to number of protein iteractions
gmt.sort_values(by = ['interactions'], ascending= False, inplace=True)

#Gain indexes of gene sets with 5 or more proteins and less than 2000 protein interactions
indices = [index for index, rowData in gmt.interactions.iteritems() if rowData >= 5 and rowData < 2000]

#filter dataframe by these indices
gmt = gmt.loc[indices]

gmt_2 = gmt.copy()

#reset index and insert a column for the description
gmt_2.insert(0, 'Description', 'KEA')
gmt_2.reset_index(inplace = True)

#Drop columns not needed in GMT and join all protein interactions by a tab
gmt_2['merged'] = ['\t'.join(x) for x in gmt_2['protein_2']]
gmt_2.drop('protein_2', axis = 1, inplace = True)
gmt_2.drop('interactions', axis = 1, inplace = True)

#create a dictionary and store in it rowData corresponding to each protein
gmt_d = dict([(key, '') for key in gmt_2.index])

# loop through rows with iterrows()
for index, rowData in gmt_2.iterrows():
    line = ('\t'.join(rowData))
    gmt_d[index] = line

## Print GMT to File and Gather Statistics

In [12]:
genes_term = gmt.interactions

len(genes_term)

325

In [13]:
sum(genes_term)

5388

In [14]:
avg_num_terms = genes_term.mean(axis = 0)
avg_num_terms

16.57846153846154

In [15]:
#For inclusion on website as a statistic, calculate 
#the total number of unique terms for the dataset
stat_df = extend.loc[indices]

stat_df.reset_index(inplace=True)

all_terms = pd.concat([stat_df.protein_1, stat_df.protein_2], axis = 0)
len(all_terms.unique())

1575

In [16]:
#Transfer tab-separated info into a new gmt file
with open('kea_ppi_filtered.gmt', 'w') as openfile:
    for index in gmt_d:
        openfile.write(str(gmt_d[index]) + '\n')